# Fine-Tune a Generative AI Model for Dialogue Summarization

In this notebook, you will fine-tune an existing LLM from Hugging Face for enhanced dialogue summarization. We will use the [OPT](https://huggingface.co/docs/transformers/model_doc/OPT) model, which provides a high quality instruction tuned model and can summarize text out of the box. To improve the inferences, you will explore a full fine-tuning approach and evaluate the results with ROUGE metrics. Then you will perform Parameter Efficient Fine-Tuning (PEFT), evaluate the resulting model and see that the benefits of PEFT outweigh the slightly-lower performance metrics.

# Table of Contents

TOC

# Convert the Ubuntu Dialogue Corpus into Q&A pairs

Select the short Q&A discussions only from the Ubuntu Dialogue Corpus.

**Acknowledgments**
https://www.kaggle.com/datasets/rtatman/ubuntu-dialogue-corpus

This dataset was ORIGINALLY collected by Ryan Lowe, Nissan Pow , Iulian V. Serban† and Joelle Pineau. It is made available here under the Apache License, 2.0. If you use this data in your work, please include the following citation:

Ryan Lowe, Nissan Pow, Iulian V. Serban and Joelle Pineau, "The Ubuntu Dialogue Corpus: A Large Dataset for Research in Unstructured Multi-Turn Dialogue Systems", SIGDial 2015. URL: http://www.sigdial.org/workshops/conference16/proceedings/pdf/SIGDIAL40.pdf

**Parser** 

https://github.com/sedthh/Open-Assistant/blob/b3a8c2479b12ea69d66487e2852b836083b7e4db/data/datasets/ubuntu_dialogue_qa/ubuntu_parser.ipynb

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LAION-AI/Open-Assistant/blob/data/datasets/ubuntu_dialogue/ubuntu_parser.ipynb)

In [1]:
# uncomment and run below lines to set up if running in colab
# !git clone https://github.com/LAION-AI/Open-Assistant.git
# %cd Open-Assistant/data/datasets/ubuntu_dialogue/
# !pip install -r requirements.txt

# Download the dataset 

In [ ]:
#!wget http://dataset.cs.mcgill.ca/ubuntu-corpus-1.0/ubuntu_dialogs.tgz . 

In [11]:
!ls ../

'Module 7'  'Ubuntu Dialogue Corpus.zip'   toc.csv
 Research    Ubuntu-dialogue-corpus


In [13]:
# global settings

FOLDER = "../Ubuntu-dialogue-corpus"  # input folder containing ubuntu dialogue csv files
SOURCE = "ubuntu-dialogue"  # source to use in the parquet for each row

In [14]:
import os
import re
import json

from tqdm import tqdm

import numpy as np
import pandas as pd

In [15]:
def load(file):
    data = pd.read_csv(file)
    data["date"] = pd.to_datetime(data["date"])
    data["id"] = data[["folder", "dialogueID"]].apply(lambda x: f"{x[0]}_{x[1].split('.tsv')[0]}" , axis=1)
    data.drop(columns=["folder", "dialogueID"], inplace=True)
    return data

data = None
for file in tqdm(os.listdir(FOLDER)):
    data = pd.concat([data, load(os.path.join(FOLDER, file))])

data.head()

100%|██████████| 3/3 [04:19<00:00, 86.66s/it] 


,date,from,to,text,id
0,2004-11-23 11:49:00+00:00,stuNNed,NaN,any ideas why java plugin takes so long to load?,301_1
1,2004-11-23 11:49:00+00:00,crimsun,stuNNed,java 1.4?,301_1
2,2004-11-23 11:49:00+00:00,stuNNed,crimsun,yes,301_1
3,2004-11-23 11:49:00+00:00,crimsun,stuNNed,java 1.5 loads _much_ faster,301_1
4,2004-11-23 11:50:00+00:00,stuNNed,crimsun,noneus: how can i get 1.5 is there a .deb some...,301_1


In [16]:
# clean up the df, remove duplicates and answers that are way too short, etc.
clean = {col:[] for col in ["INSTRUCTION", "RESPONSE", "SOURCE", "METADATA"]}

for name, group in tqdm(data.groupby("id")):
    if len(group) < 3 or len(group) > 5:  # 3, 4, 5 len
        continue  # back and forth will most likely not be parsed correctly
    
    group.sort_values(by=["date"], ascending=True, inplace=True)
    instruction = str(group["text"].values[0]).strip()
    insturction_user = group["from"].values[0]
    if not instruction or pd.isna(instruction) or len(instruction) < 12:
        continue
    if not re.findall(r'(?i)(?:\?|what|who|where|why|when|how|whose|explain|tell|does|way|can|know|able|best|recommend)', instruction):
        continue  # probably not a question
    
    all_recipients = "|".join([re.escape(item) for item in set(group["to"].tolist() + group["from"].tolist()) if pd.notna(item)])
    response = None
    response_user = None
    for _, row in group.iterrows():
        if row["to"] == insturction_user:
            candidate = str(row["text"]).strip()
            if not row["text"] or pd.isna(row["text"]) or re.findall(r'(?i)^(yes|yep|yeah|no|nah|nope|sure|yes\s*sir)\W*$', candidate):
                continue  # answer is not expressive
            if len(candidate) < 3:
                continue  # too short
            if re.findall(r'(?i)(?:wrong|of.*?topic|else\s*where|ask.+?in|\#\w+|google|you.+?mean)', candidate):
                continue  # probably off topic
            if re.findall(r'\b(' + all_recipients + r')\b', candidate):
                continue  # answer includes user name(s)
            response = candidate
            response_user = row["from"]
        elif response_user is not None and row["to"] == response_user and row["from"] == insturction_user:
            if re.findall(r'(?i)(?:thank|thx|works|working|great)', str(row["text"])):
                clean["INSTRUCTION"].append(instruction)
                clean["RESPONSE"].append(response)
                clean["SOURCE"].append(SOURCE)
                clean["METADATA"].append(json.dumps({
                    "user_question": insturction_user,
                    "user_answer": response_user
                }))
                break
             

100%|██████████| 1852868/1852868 [04:01<00:00, 7657.74it/s] 


In [17]:
clean = pd.DataFrame(clean)
clean.sort_values(by="RESPONSE", key=lambda x: x.str.len(), inplace=True, ascending=False)
clean.drop_duplicates(subset=["INSTRUCTION"], inplace=True)
clean.sort_index(inplace=True)
clean.head()

,INSTRUCTION,RESPONSE,SOURCE,METADATA
0,"hi, is there a CLI command to roll back any up...",your recourse is to re-install fresh the older...,ubuntu-dialogue,"{""user_question"": ""edd"", ""user_answer"": ""n8tus..."
1,A LiveCD iso can be burned to a DVD-R and run ...,"I hope so, or the custom DVDs I've done are wo...",ubuntu-dialogue,"{""user_question"": ""usrl"", ""user_answer"": ""Ghos..."
2,"hello, is there a way to adjust gamma settings...",for me i have my nvidia settings manager and i...,ubuntu-dialogue,"{""user_question"": ""nucco_"", ""user_answer"": ""sp..."
4,does ubuntu come with a firewall by default?,no iptables rule is loaded by deault on ubuntu,ubuntu-dialogue,"{""user_question"": ""aeleon"", ""user_answer"": ""er..."
5,Can someone tell me howto get rid of Google Ch...,sudo dpkg -l |grep -i chrom ----> sudo apt-get...,ubuntu-dialogue,"{""user_question"": ""frold"", ""user_answer"": ""shi..."


In [18]:
print(f"Retrieved {len(clean) / len(data['id'].unique()) * 100.:.2f}% of all questions ({len(clean)})")  # 19921

Retrieved 0.87% of all questions (16181)


In [20]:
for index, row in clean.iterrows():
    print("Q >", row["INSTRUCTION"])
    print("A >", row["RESPONSE"])
    print()
    if index > 20:
        break

Q > hi, is there a CLI command to roll back any updates/upgrades I made recently?
A > your recourse is to re-install fresh the older version

Q > A LiveCD iso can be burned to a DVD-R and run with no problems, right?
A > I hope so, or the custom DVDs I've done are worthless. ;)

Q > hello, is there a way to adjust gamma settings in totem? my videos aren't playing with the correct colours
A > for me i have my nvidia settings manager and i change the video gamma settings from there...

Q > does ubuntu come with a firewall by default?
A > no iptables rule is loaded by deault on ubuntu

Q > Can someone tell me howto get rid of Google Chrome? Im not able to uninstall it...
A > sudo dpkg -l |grep -i chrom ----> sudo apt-get remove 'on what appears'

Q > why is there competition between linux distros? They are all free...
A > not really ontopic here but just personal preference

Q > wow. for the life of me i can never remember this command. whats the command that outputs your ati hardare info

In [21]:
clean.to_csv('clean.csv', index=False)

<a name='1'></a>
## 1 - Set up Kernel, Load Required Dependencies, Dataset and LLM

<a name='1.1'></a>
### 1.1 - Set up Kernel and Required Dependencies

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
# !pip install --upgrade pip
# !pip install --disable-pip-version-check \
#     torch==1.13.1 \
#     torchdata==0.5.1 --quiet

# !pip install \
#     transformers==4.27.2 \
#     datasets==2.11.0 \
#     evaluate==0.4.0 \
#     rouge_score==0.1.2 \
#     loralib==0.1.1 \
#     peft==0.3.0 --quiet

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, AutoModelForCausalLM
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import torch.nn as nn
import bitsandbytes as bnb
import pandas as pd
from datasets import DatasetDict, Dataset
import torch
from transformers import AutoTokenizer, T5ForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from evaluate import load

<a name='1.2'></a>
### 1.2 - Load Dataset and LLM

You are going to continue experimenting with the [ubuntu_dialogs_corpus](https://huggingface.co/datasets/ubuntu_dialogs_corpus) Hugging Face dataset. 

Ubuntu Dialogue Corpus, a dataset containing almost 1 million multi-turn dialogues, with a total of over 7 million utterances and 100 million words. This provides a unique resource for research into building dialogue managers based on neural language models that can make use of large amounts of unlabeled data. The dataset has both the multi-turn property of conversations in the Dialog State Tracking Challenge datasets, and the unstructured nature of interactions from microblog services such as Twitter.

### Using v2 of the ubuntu_dialogs_corpus

preprocesed during download for steming and lammetization   
Source https://github.com/rkadlec/ubuntu-ranking-dataset-creator  
!git clone https://github.com/rkadlec/ubuntu-ranking-dataset-creator.git .  
./ubuntu-ranking-dataset-creator/src/./generate.sh   
!cp ./ubuntu-ranking-dataset-creator/src/*.csv  ../Datasets/Ubuntu-corpus  

In [9]:
data = load_dataset("csv", data_files = "./clean.csv")

Found cached dataset csv (/home/jupyter/.cache/huggingface/datasets/csv/default-2e0c7a9fed0024c6/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
data

DatasetDict({
    train: Dataset({
        features: ['INSTRUCTION', 'RESPONSE', 'SOURCE', 'METADATA'],
        num_rows: 16181
    })
})

In [11]:
# Rename features
data['train'] = data['train'].rename_column('INSTRUCTION', 'context') 
data['train'] = data['train'].rename_column('RESPONSE', 'response')

# Drop features
data['train'] = data['train'].remove_columns(['SOURCE', 'METADATA'])

In [13]:
data_corpus = pd.DataFrame(data['train'])

In [26]:
data_corpus.head()

,context,response
0,"hi, is there a CLI command to roll back any up...",your recourse is to re-install fresh the older...
1,A LiveCD iso can be burned to a DVD-R and run ...,"I hope so, or the custom DVDs I've done are wo..."
2,"hello, is there a way to adjust gamma settings...",for me i have my nvidia settings manager and i...
3,does ubuntu come with a firewall by default?,no iptables rule is loaded by deault on ubuntu
4,Can someone tell me howto get rid of Google Ch...,sudo dpkg -l |grep -i chrom ----> sudo apt-get...


## Sample questions with responses in dataset

In [8]:
sample_questions=[
    "does ubuntu come with a firewall by default?",
    "Can grub-install work with ext3?",
    "Is there a CLI command to roll back updates?",
    "How do I get rid of rid of Google Chrome?"
]
    

# Experimentation v1

Experimentation: Goal is to create a support bot

# Load the Model

In [16]:
model_name='facebook/opt-6.7b'

original_model = AutoModelForCausalLM.from_pretrained(model_name, 
                                                       device_map="auto",
                                                       load_in_8bit=True,
                                                       torch_dtype=torch.float16 )
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Test the Model with Zero Shot Inferencing

In [5]:
original_model.eval()

In [6]:
def test_model_with_zero_shot(question, model_eval):
    test_inputs = tokenizer(question, return_tensors="pt")

    ## Asign Inputs to GPU|
    with torch.cuda.amp.autocast():
        test_inputs = test_inputs.to('cuda')
        test_outputs = model_eval.generate(**test_inputs,
                            max_new_tokens=64,
                            repetition_penalty=2.1,
                            length_penalty = 1.8,
                            temperature = 1)
                           
    return tokenizer.decode(test_outputs[0], skip_special_tokens=True, padding_side='left')
    

In [28]:
for question in sample_questions:
    print("Question:",question,"\n")
    print("Model Output:",test_model_with_zero_shot(question, original_model),"\n")
    

/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Question: does ubuntu come with a firewall by default? 

Model Output: does ubuntu come with a firewall by default?
Yes, it comes pre-installed.  I'm not sure if you can disable the built in one though...I've never tried to do that before but maybe someone else knows for certain! :)

 Edit: Just checked and yes - there is an option under System > Administration (or something like this) called 

Question: Can grub-install work with ext3? 

Model Output: Can grub-install work with ext3?
I don't think so.  I've never tried it, but the documentation says that you need to use EXT4 for GRUB2 and XFS or BTRfs (or something like this) if using a filesystem other than FAT32/NTFS on Linux systems... which is what Grubby uses 

Question: Is there a CLI command to roll back updates? 

Model Output: Is there a CLI command to roll back updates?
I don't think so.  I've never seen one, but it would be nice if you could do that! :)

 You can always just uninstall the update and then reinstall from scra

# Model Preprocess

In [29]:
for param in original_model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

original_model.gradient_checkpointing_enable()  # reduce number of stored activations
original_model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
original_model.lm_head = CastOutputToFloat(original_model.lm_head)

In [32]:
max_seq_len = 762

def tokenize_function(example):
    #start_prompt = 'Act like a chat agent in a turn conversation similar to this dialogue between two individuals where one individual is looking to resolve Ubuntu technical issues. For every new request, provide a response\n\n'
    start_prompt = """
    Technical Question: \n
    """
    end_prompt = '\n\nCorrect response to technical question: \n '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["context"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", max_length=max_seq_len, truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["response"],padding="max_length", max_length=max_seq_len, truncation=True, return_tensors="pt").input_ids
    
    return example

tokenized_datasets = data.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['response', 'context'])

Map:   0%|          | 0/16181 [00:00<?, ? examples/s]

In [34]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")


print(tokenized_datasets)
print(tokenized_datasets['train'][:1])
# print(tokenized_datasets['train'][:1]['input_ids'][0])
# print(tokenized_datasets['train'][:1]['labels'][0])

Shapes of the datasets:
Training: (16181, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 16181
    })
})
{'input_ids': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

### 3.1 - Setup the PEFT/LoRA model for Fine-Tuning

You need to set up the PEFT/LoRA model for fine-tuning with a new layer/parameter adapter. Using PEFT/LoRA, you are freezing the underlying LLM and only training the adapter. Have a look at the LoRA configuration below. Note the rank (`r`) hyper-parameter, which defines the rank/dimension of the adapter to be trained.

In [35]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# Define LoRA Config
lora_config = LoraConfig(
 r=32,
 lora_alpha=32,
 target_modules=["q_proj", "v_proj"],
 lora_dropout=0.01,
 bias="none",
 task_type="CAUSAL_LM"
)

### prepare int-8 model for training

In [ ]:
# model_int8 = prepare_model_for_int8_training(original_model)

### add LoRA adaptor

In [36]:
model_int8_lora = get_peft_model(original_model, lora_config)
model_int8_lora.print_trainable_parameters()

trainable params: 16777216 || all params: 6675251200 || trainable%: 0.2513346014603915


### 3.2 - Train PEFT Adapter

Define training arguments and create `Trainer` instance.

In [37]:
import transformers

output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'


peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    max_steps=100,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=2,
    logging_steps=20, 
    fp16=True,
    save_strategy = "no"
)
    
trainer = Trainer(
    model=model_int8_lora,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

original_model.config.use_cache = False

In [38]:
# Train model
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
20,5.129600
40,4.538800
60,2.833500
80,1.836000
100,1.674500


TrainOutput(global_step=100, training_loss=3.2024721908569336, metrics={'train_runtime': 642.7601, 'train_samples_per_second': 1.245, 'train_steps_per_second': 0.156, 'total_flos': 2.36315351973888e+16, 'train_loss': 3.2024721908569336, 'epoch': 0.05})

In [44]:
# Save our LoRA model & tokenizer results
peft_model_id="results-opt"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)


('results-opt/tokenizer_config.json',
 'results-opt/special_tokens_map.json',
 'results-opt/vocab.json',
 'results-opt/merges.txt',
 'results-opt/added_tokens.json',
 'results-opt/tokenizer.json')

### Evaluate & run Inference with LoRA OPT

We are going to use evaluate library to evaluate the rogue score. We can run inference using PEFT and transformers. For our OPT large model

In [12]:
from peft import PeftModel, PeftConfig
# Load peft config for pre-trained checkpoint etc. 

peft_model_id = "results-opt"
config = PeftConfig.from_pretrained(peft_model_id)

# load base LLM model and tokenizer
base_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, device_map="auto")

## Load the Lora model
test_model = PeftModel.from_pretrained(base_model, peft_model_id, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
test_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): OPTForCausalLM(
      (model): OPTModel(
        (decoder): OPTDecoder(
          (embed_tokens): Embedding(50272, 4096, padding_idx=1)
          (embed_positions): OPTLearnedPositionalEmbedding(2050, 4096)
          (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
          (layers): ModuleList(
            (0): OPTDecoderLayer(
              (self_attn): OPTAttention(
                (k_proj): Linear(in_features=4096, out_features=4096, bias=True)
                (v_proj): Linear(
                  in_features=4096, out_features=4096, bias=True
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.01, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=32, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_featu

# Testing with sample questions

In [15]:
for question in sample_questions:
    print("Question:",question,"\n")
    print("Model Output:",test_model_with_zero_shot(question, test_model),"\n")

Question: does ubuntu come with a firewall by default? 

Model Output: does ubuntu come with a firewall by default?
Yes, it does.  I'm not sure what you're trying to do here... but if your goal is "to get rid of the built-in firewalls", then no - Ubuntu doesn't have one either (unless there's some other reason for this post).. and even so that would be 

Question: Can grub-install work with ext3? 

Model Output: Can grub-install work with ext3?
I don't think so.  I'm pretty sure it's a 4 partition thing, and the installer is only for 2 partitions (ext2/4) or 3).

 If you want to try installing ubuntu on an external hard drive then booting from that way...it works fine! :) good luck 

Question: Is there a CLI command to roll back updates? 

Model Output: Is there a CLI command to roll back updates?
sudo apt-get distupgrade -y  This will downgrade your system. You can then upgrade again after you're done with this, but it's not recommended as the process is very easy and quick if that fa

https://medium.com/@tom_21755/understanding-causal-llms-masked-llm-s-and-seq2seq-a-guide-to-language-model-training-d4457bbd07fa

# Serve using Gradio

In [61]:
print(serving("how do you mount a cd"))

Question: how do you mount a cd 

Model Output: how do you mount a cd drive to the back of your case?
I used velcro and zip ties. I also have it mounted on my desk with an external power supply so that i can just plug in when needed, but its not necessary for this build as there is no need at all since im using sata drives only :)  Edit 

None


In [16]:
import gradio as gr

def serving(sample):
    return test_model_with_zero_shot(sample,test_model)

demo = gr.Interface(fn=serving, inputs="text", outputs="text")
    
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://22f7c53c8d95bf6216.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
